In [45]:
import pandas as pd

In [8]:
df = pd.read_json('Sarcasm_Headlines_Dataset.json', lines = True)

In [9]:
headlines

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [17]:
# importing all the libraries:

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
# tokenizing and padding the data: 

# Split the dataset into training and testing sets
train_texts, test_texts, y_train, y_test = train_test_split(df['headline'], df['is_sarcastic'], test_size=0.2, random_state=42)

# Tokenize and pad sequences
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

max_sequence_length = 100  # Adjust as needed
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')


In [21]:
# building and training the LSTM model: 

embedding_dim = 16  # Adjust as needed

vocab_size = 10000
max_length = 100


model = Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
epochs = 5 
model.fit(train_padded, y_train, epochs=epochs, validation_data=(test_padded, y_test))


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 24)                408       
                                                                 
 dense_4 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
668/668 [==============================] - 3s 3ms/step - loss: 0.6748 - accuracy: 0.5740 - val_loss: 0.6

In [23]:
# making predictions: 

# Make predictions on new data
new_texts = ["Your new sarcastic or non-sarcastic news texts here", "Another news text"]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

predictions = model.predict(new_padded)


1/1 [==============================] - 0s 33ms/step


In [24]:
predictions

array([[0.0902847 ],
       [0.47782803]], dtype=float32)

In [33]:

ground_truth_labels = df['is_sarcastic'].values

# Converting model predictions to binary values based on a threshold
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)

# Comparing predictions with ground truth labels
correct_predictions = (binary_predictions == ground_truth_labels)

# Printing the results
for text, prediction, ground_truth, is_correct in zip(new_texts, binary_predictions, ground_truth_labels, correct_predictions):
    print(f"Text: {text}")
    print(f"Predicted: {prediction}, Actual: {ground_truth}, Correct: {is_correct}")
    print("\n")


Text: Your new sarcastic or non-sarcastic news texts here
Predicted: [0], Actual: 0, Correct: [ True  True False ...  True  True  True]


Text: Another news text
Predicted: [0], Actual: 0, Correct: [ True  True False ...  True  True  True]


